### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

### Drop Null target records

In [5]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [6]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [7]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [8]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [9]:
features_list=[
'statedIncome',
'deviation_from_median_mean_income#last_365_Days',
'deviation_from_mean_income#last_365_Days',
'deviation_from_median_mean_income#last_7_days',
'yearOfBirth_mean',
'median_mean_income#last_180_Days',
'deviation_from_mean_age#last_365_Days',
'yearOfBirth_median',
'authorized_capital',
'age',
'paidup_capital',
'avg_income#last_365_Days',
'mean_loan_amount#last_365_Days',
'loan_amount_to_income#last_365_Days',
'year_of_registration',
'avg_income#last_7_days',
 'month_year',
 '60_in_15mob']

In [10]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [11]:
oot_df = oot_df1[features_list]

In [12]:
model_df.shape, oot_df.shape

((101984, 18), (17343, 18))

In [13]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [14]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [15]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [16]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [17]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3033486/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [18]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3033486/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [19]:
X_oot.shape,y_oot.shape

((17343, 16), (17343,))

### Split Model dataset into Train and Test

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [21]:
X_train.shape, X_test.shape

((61190, 16), (40794, 16))

In [22]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 16) (12238, 16) (40794, 16)


In [23]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [24]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.005,
    'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,
    'lambda': 4.7,
    'alpha': 1.8,
    'random_state': 42  
}


evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.52153	eval-auc:0.52552
[10]	train-auc:0.55835	eval-auc:0.56564
[20]	train-auc:0.56483	eval-auc:0.56783
[30]	train-auc:0.56647	eval-auc:0.57121
[40]	train-auc:0.56536	eval-auc:0.57270
[50]	train-auc:0.56596	eval-auc:0.57331
[60]	train-auc:0.56585	eval-auc:0.57312
[70]	train-auc:0.56610	eval-auc:0.57432
[80]	train-auc:0.56692	eval-auc:0.57448
[90]	train-auc:0.56678	eval-auc:0.57403
[100]	train-auc:0.56691	eval-auc:0.57460
[110]	train-auc:0.56745	eval-auc:0.57517
[120]	train-auc:0.56779	eval-auc:0.57548
[130]	train-auc:0.56812	eval-auc:0.57528
[140]	train-auc:0.56820	eval-auc:0.57504
[150]	train-auc:0.56811	eval-auc:0.57484
[160]	train-auc:0.56819	eval-auc:0.57496
[170]	train-auc:0.56851	eval-auc:0.57479
[174]	train-auc:0.56851	eval-auc:0.57485


In [25]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.57, Gini: 0.14, KS: 0.10
Eval AUC: 0.57, Gini: 0.15, KS: 0.11
Test AUC: 0.56, Gini: 0.12, KS: 0.10
OOT AUC: 0.56, Gini: 0.12, KS: 0.10


In [26]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.57, Gini: 0.1370, KS: 0.0974
Test AUC: 0.56, Gini: 0.1231, KS: 0.1007
OOT AUC: 0.56, Gini: 0.1218, KS: 0.1027


In [27]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                            Feature  Importance
0                                      statedIncome        75.0
4                                  yearOfBirth_mean        73.0
1   deviation_from_median_mean_income#last_365_Days        55.0
2          deviation_from_mean_income#last_365_Days        40.0
9                                               age        38.0
3     deviation_from_median_mean_income#last_7_days        37.0
7                                yearOfBirth_median        33.0
6             deviation_from_mean_age#last_365_Days        32.0
5                  median_mean_income#last_180_Days        30.0
13              loan_amount_to_income#last_365_Days        24.0
11                         avg_income#last_365_Days        17.0
14                             year_of_registration        16.0
15                           avg_income#last_7_days        16.0
8                                authorized_capital        14.0
12                   mean_loan_amount#la

In [28]:
feature_importance_df

Feature  Importance
0                                      statedIncome        75.0
4                                  yearOfBirth_mean        73.0
1   deviation_from_median_mean_income#last_365_Days        55.0
2          deviation_from_mean_income#last_365_Days        40.0
9                                               age        38.0
3     deviation_from_median_mean_income#last_7_days        37.0
7                                yearOfBirth_median        33.0
6             deviation_from_mean_age#last_365_Days        32.0
5                  median_mean_income#last_180_Days        30.0
13              loan_amount_to_income#last_365_Days        24.0
11                         avg_income#last_365_Days        17.0
14                             year_of_registration        16.0
15                           avg_income#last_7_days        16.0
8                                authorized_capital        14.0
12                   mean_loan_amount#last_365_Days        13.0
10                                   paidup_capital        12.0

In [29]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.162786         0.151185        0.186047        0.145100   4896   
9          0.124617         0.142923        0.145099        0.140883   4895   
8          0.120531         0.138861        0.140882        0.136830   4895   
7          0.113177         0.135157        0.136827        0.133829   4895   
6          0.110317         0.132994        0.133828        0.132309   4895   
5          0.104392         0.131738        0.132308        0.131208   4895   
4          0.096425         0.130752        0.131207        0.130287   4895   
3          0.090092         0.129824        0.130287        0.129341   4895   
2          0.080695         0.128861        0.129341        0.128366   4895   
1          0.075980         0.127728        0.128366        0.126205   4896   

        label_good  label_bad  
Decile                         
10            4099        797  
9             4285        610  
8             4305        590  
7             4341        554  
6             4355        540  
5             4384        511  
4             4423        472  
3             4454        441  
2             4500        395  
1             4524        372

In [30]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.157843         0.151270        0.187133        0.145178   4080   
9          0.126992         0.142990        0.145177        0.140911   4079   
8          0.120618         0.138841        0.140910        0.136773   4079   
7          0.130147         0.135092        0.136772        0.133789   4080   
6          0.110566         0.132984        0.133788        0.132285   4079   
5          0.100270         0.131712        0.132284        0.131182   4079   
4          0.096324         0.130711        0.131182        0.130244   4080   
3          0.095857         0.129779        0.130244        0.129299   4079   
2          0.080902         0.128828        0.129299        0.128361   4079   
1          0.086765         0.127734        0.128360        0.126205   4080   

        label_good  label_bad  
Decile                         
10            3436        644  
9             3561        518  
8             3587        492  
7             3549        531  
6             3628        451  
5             3670        409  
4             3687        393  
3             3688        391  
2             3749        330  
1             3726        354

In [31]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.149280         0.152488        0.185698        0.145795   1735   
9          0.129181         0.143308        0.145787        0.140853   1734   
8          0.115340         0.138840        0.140853        0.137068   1734   
7          0.126298         0.135416        0.137068        0.134072   1734   
6          0.096309         0.133241        0.134071        0.132538   1734   
5          0.092219         0.132010        0.132537        0.131497   1735   
4          0.088235         0.131030        0.131497        0.130567   1734   
3          0.098616         0.130079        0.130566        0.129588   1734   
2          0.081315         0.129059        0.129587        0.128553   1734   
1          0.082997         0.127856        0.128552        0.126066   1735   

        label_good  label_bad  
Decile                         
10            1476        259  
9             1510        224  
8             1534        200  
7             1515        219  
6             1567        167  
5             1575        160  
4             1581        153  
3             1563        171  
2             1593        141  
1             1591        144

In [32]:
#Save the Model
xgb_model.save_model('LeoPayu_LA_60_in_15mob.json')
print('Model Saved')

Model Saved


In [33]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_LA_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_LA_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_LA_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_LA_Oot_Pred.csv',index=False)
